In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
def corr2d(X, K): #@save
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h,j:j+w] * K).sum()
    return Y       

In [ ]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
print(X)
print(K)

In [ ]:
corr2d(X, K)

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(size=kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

In [ ]:
K = torch.tensor([[1.0, -1.0]])

In [ ]:
Y = corr2d(X, K)

In [ ]:
Y

In [ ]:
conv2 = nn.LazyConv2d(out_channels=1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

Y_hat = conv2(X)
conv2.zero_grad()
l = (Y_hat - Y) ** 2

l.sum().backward()
print(conv2.weight.grad)

In [ ]:
conv2d = nn.LazyConv2d(out_channels=1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2
Y_hat = conv2(X)
for i in range(50):
    Y_hat = conv2(X)
    l = (Y_hat - Y) ** 2
    conv2.zero_grad()
    l.sum().backward()
    #Update the kernel
    conv2.weight.data -= lr * conv2.weight.grad
    if (i + 1)%2 == 0:
        print(f"epoch: {i + 1}, loss: {l.sum()}")
print(conv2.weight.data)

In [26]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])
conv2d = nn.LazyConv2d(1, kernel_size=3,padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [27]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), stride=2, padding=1)
comp_conv2d(conv2d, X).shape

torch.Size([4, 3])